# **Import Libraries**

In [ ]:
!pip install azure-storage-blob

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from bs4 import BeautifulSoup
import os
import re
import requests
from azure.storage.blob import BlobServiceClient
import json
import io
import warnings
warnings.filterwarnings('ignore')
from sqlalchemy import create_engine

# **Data Extraction**

In [ ]:
def load_config_azure_professor(config_path="config.json"):
    """Load the Azure configuration parameters from the config.json file."""
    with open(config_path, "r", encoding="utf-8") as config_file:
        config = json.load(config_file)
    return config["AZURE_CONNECTION_STRING"], config["CONTAINER_NAME"]


def load_config_azure_pat(config_path="config.json"):
    """Load the Azure configuration parameters from the config.json file."""
    with open(config_path, "r", encoding="utf-8") as config_file:
        config = json.load(config_file)
    return config["AZURE_CONNECTION_STRING_PAT"], config["CONTAINER_NAME_PAT"]


AZURE_CONNECTION_STRING, CONTAINER_NAME = load_config_azure_professor()
AZURE_CONNECTION_STRING_PAT, CONTAINER_NAME_PAT = load_config_azure_pat()

blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)
folder_name = 'flights/'

blob_service_client_pat = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING_PAT)
container_client_pat = blob_service_client_pat.get_container_client(CONTAINER_NAME_PAT)
folder_name_pat = 'flights'

In [ ]:
for blob in container_client.list_blobs(name_starts_with=folder_name):
    # Create a blob client for the destination
    blob_client = blob_service_client_pat.get_blob_client(container=CONTAINER_NAME_PAT,blob=folder_name_pat + blob.name)

    # Read the source blob and upload to the new storage
    source_blob_data = container_client.download_blob(blob.name).readall()
    blob_client.upload_blob(source_blob_data, overwrite=True)